## Importing Dataset and Libraries

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import random

In [3]:
df = pd.read_csv('Dry Bean Dataset.csv')
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


## Preprocessing

In [4]:
df['Class'] = LabelEncoder().fit_transform(df['Class'])

x = df.drop('Class', axis=1)
y = df['Class']

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


## Creating samples

In [5]:
samples = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, stratify=y, random_state=i)
    samples.append((X_train, X_test, y_train, y_test))


## Applying SVM

In [6]:
from tqdm.notebook import tqdm

results = []
all_convergences = []

# Outer loop progress: 10 samples
for sample_id in tqdm(range(10), desc="Samples"):
    X_train, X_test, y_train, y_test = train_test_split(
        x_scaled, y, test_size=0.3, stratify=y, random_state=sample_id)

    best_acc = 0
    best_params = {}
    acc_list = []

    # Inner loop progress: 100 iterations for parameter search
    for i in tqdm(range(100), desc=f"Optimizing Sample {sample_id+1}", leave=False):
        kernel = random.choice(['linear', 'poly', 'rbf', 'sigmoid'])
        C = random.choice([0.1, 1, 10, 100])
        gamma = random.choice(['scale', 0.01, 0.1, 1])
        degree = random.choice([2, 3, 4]) if kernel == 'poly' else 3

        model = SVC(kernel=kernel, C=C, gamma=gamma, degree=degree)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        acc_list.append(acc)

        if acc > best_acc:
            best_acc = acc
            best_params = {
                'kernel': kernel,
                'C': C,
                'gamma': gamma,
                'degree': degree
            }

    all_convergences.append((sample_id, acc_list))
    results.append({
        'Sample': sample_id + 1,
        'Best Accuracy': best_acc,
        'Kernel': best_params['kernel'],
        'C': best_params['C'],
        'Gamma': best_params['gamma'],
        'Degree': best_params['degree']
    })

# Save results table
df_results = pd.DataFrame(results)


Samples:   0%|          | 0/10 [00:00<?, ?it/s]

Optimizing Sample 1:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 2:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 3:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 4:   0%|          | 0/100 [00:00<?, ?it/s]

...

Optimizing Sample 6:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 7:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 8:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 9:   0%|          | 0/100 [00:00<?, ?it/s]

Optimizing Sample 10:   0%|          | 0/100 [00:00<?, ?it/s]